In [1]:
import pyensembl
from pyensembl import Genome
import pandas as pd
import csv
import pymysql
import operator
import time
import datetime
from termcolor import colored
import math

#### Import reference GTF to retrieve genes names by coordinates

In [2]:
# wget ftp://ftp.ensembl.org/pub/release-98/gtf/homo_sapiens/Homo_sapiens.GRCh38.98.gtf.gz
data = Genome(reference_name='GRCh38', annotation_name='98', gtf_path_or_url='REF/Homo_sapiens.GRCh38.98.gtf')

In [3]:
# only if index is not built already -> saved in current folder
# data.index()

In [3]:
# Pysensembl function to retrieve gene name from coordinates
data.gene_names_at_locus(contig=1, position=630755, end=634922, strand='+') 

['MTATP6P1', 'MTATP8P1', 'MTCO1P12', 'MTCO2P12', 'MTCO3P12']

In [ ]:
# Pyensembl function retrieves gene name from ID
data.gene_name_of_gene_id('ENSG00000260500')

## RSEM TXIMPORT - get genes symbols

### DESeq2

In [7]:
id2genes_df = pd.read_csv("RSEM_TEMP/rsem_geneID_dds.txt", sep='\t', skiprows=(0), header=(0))
i = 0
j = 0
gn_list = []
for idx, row in id2genes_df.iterrows():
    gname = ''
    gname = data.gene_name_of_gene_id(row['GENE_ID'].split('.')[0])
    if gname != '':
        i+=1
    else:
        j+=1
    gn_list.append(gname)
id2genes_df['GENE_NAME'] = gn_list
id2genes_df.to_csv("RSEM_TEMP/rsem_geneID_dds_OK.txt", sep='\t', index=False)
print(f'Filled {i} genes symbols, still {j} remain unknown')

Filled 26279 genes symbols, still 0 remain unknown


### LIMMA

In [9]:
id2genes_df = pd.read_csv("RSEM_TEMP/LIMMA_rsem_id2genes.txt", sep='\t', skiprows=(0), header=(0))
i = 0
j = 0
gn_list = []
for idx, row in id2genes_df.iterrows():
    gname = ''
    gname = data.gene_name_of_gene_id(row['GENE_ID'])
    if gname != '':
        i+=1
    else:
        j+=1
    gn_list.append(gname)
id2genes_df['GENE_NAME'] = gn_list
id2genes_df.to_csv("RSEM_TEMP/LIMMA_rsem_id2genes_OK.txt", sep='\t', index=False)
print(f'Filled {i} genes symbols, still {j} remain unknown')

Filled 22875 genes symbols, still 0 remain unknown


# RSEM matrices preprocessing

In [36]:
df_rsem_mat = pd.read_csv('RSEM_RES/GeneMat.de.txt', sep='\t', skiprows=(0), header=(0))

In [37]:
df_rsem_mat.head()

,PPEE,PPDE,PostFC,RealFC,C1Mean,C2Mean
ENSG00000087494,0.0,1.0,0.006978,0.000453,0.00000,22.071668
ENSG00000106536,0.0,1.0,133.230978,2051.903969,20.50904,0.000000
ENSG00000109956,0.0,1.0,0.009233,0.000600,0.00000,16.643234
ENSG00000115590,0.0,1.0,0.005795,0.000376,0.00000,26.609493
ENSG00000124092,0.0,1.0,0.005281,0.000342,0.00000,29.214707


In [38]:
ensembl_rsem = list(df_rsem_mat.index)

In [39]:
# Query pyensembl to get gene symbols (if exists and unique)
genes_list = []
for accession in ensembl_rsem:
    name = data.gene_name_of_gene_id(accession)
    genes_list.append(name)

In [40]:
len(ensembl_rsem), len(genes_list)

(602, 602)

In [41]:
df_rsem_mat.index = genes_list

In [44]:
df_rsem_mat.to_csv('RSEM_RES/GeneMat.de.txt', sep='\t', index=True)

In [45]:
## second matrix from Jupyter

In [46]:
df_rsem_mat = pd.read_csv('RSEM_RES/EBSeq_PPMat.txt', sep='\t', skiprows=(0), header=(0))
ensembl_rsem = list(df_rsem_mat.index)
genes_list = []
for accession in ensembl_rsem:
    name = data.gene_name_of_gene_id(accession)
    genes_list.append(name)
df_rsem_mat.index = genes_list
df_rsem_mat.to_csv('RSEM_RES/EBSeq_PPMat.txt', sep='\t', index=True)

#### operation to map genes names to each isoform for EBSeq

In [51]:
# load any genes.results file
df_rsem_genes_res = pd.read_csv('CTH/RSEM/RNA2498/RS_RNA2498.genes.results', sep='\t', skiprows=(0), header=(0))

In [52]:
# extract genes names list
genes_list = list(df_rsem_genes_res['gene_id'])
len(genes_list)

60623

In [53]:
genes_list[:3]

['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419']

In [54]:
# extract transcripts names list
transcripts_list = list(df_rsem_genes_res['transcript_id(s)'])
len(transcripts_list)

60623

In [55]:
# create a list of lists of isoforms names, each mapping to a gene name
trans_list = []
for row in transcripts_list:
    sub_list = row.split(',')
    trans_list.append(sub_list)
len(trans_list)

60623

In [56]:
trans_list[:3]

[['ENST00000373020',
  'ENST00000494424',
  'ENST00000496771',
  'ENST00000612152',
  'ENST00000614008'],
 ['ENST00000373031', 'ENST00000485971'],
 ['ENST00000371582',
  'ENST00000371584',
  'ENST00000371588',
  'ENST00000413082',
  'ENST00000466152',
  'ENST00000494752']]

In [59]:
# dictionary of gene names (key) each containing a list of isoform names (value)
names_dic = {}
for i in range(len(trans_list)):
    names_dic[genes_list[i]] = trans_list[i]

In [69]:
len(names_dic)

60623

In [63]:
# load the isoform names exported from R
df_iso_names = pd.read_csv('RSEM_RES/isoforms_names.csv', sep=',', skiprows=(0), header=(0))

In [65]:
iso_names_list = list(df_iso_names['x'])

In [66]:
iso_names_list[:3]

['ENST00000371582', 'ENST00000371584', 'ENST00000371588']

In [80]:
len(iso_names_list)

110310

In [78]:
# create a list of genes names that correspond to their isoforms names
# by looking into the reference dictionary
map_genes_list = []
for trans in iso_names_list:
    for key, value in names_dic.items():
        if trans in value:
            map_genes_list.append(key)
            break

In [79]:
len(map_genes_list)

110310

In [86]:
# exporting the list for further processing in R
with open('RSEM_RES/genes_names_map.txt', 'w') as outfile:
    for line in map_genes_list:
        outfile.write(line + '\n')

# LIMMA Post-processing

In [20]:
limma_df = pd.read_csv('RSEM_RES/Limma_results_2.txt', sep='\t', skiprows=(0), header=(0))

In [21]:
limma_df

,A,Coef.C,Coef.T,t.C,t.T,p.value.C,p.value.T,Res.C,Res.T
ENSG00000000419,3.513789,3.395900,3.620530,21.349769,23.493761,2.050865e-14,3.959232e-15,1,1
ENSG00000000457,3.820349,3.779152,3.878203,23.238564,23.547194,4.779373e-15,3.807343e-15,1,1
ENSG00000000460,3.175358,3.023527,3.338193,12.608922,14.843542,1.340610e-10,9.384541e-12,1,1
ENSG00000000938,7.038729,7.208707,6.865186,31.911077,30.737864,1.953403e-17,3.751706e-17,1,1
ENSG00000001036,3.013023,2.919475,3.083917,10.530285,11.307944,2.295564e-09,7.568115e-10,1,1
...,...,...,...,...,...,...,...,...,...
ENSG00000287978,1.631122,1.799707,1.476898,2.491650,1.397367,1.142110e-02,8.975628e-02,1,0
ENSG00000287979,3.887545,4.438174,3.410290,6.680318,3.779288,1.544929e-06,7.011668e-04,1,1
ENSG00000288156,3.682152,3.661180,3.758929,11.411621,11.488687,6.556915e-10,5.897489e-10,1,1
ENSG00000288380,2.632428,2.735178,2.506864,8.831436,7.073516,3.242462e-08,7.272774e-07,1,1


In [22]:
ensembl_list = list(limma_df.index)

In [24]:
# Query pyensembl to get gene symbols (if exists and unique)
genes_list = []
for accession in ensembl_list:
    name = data.gene_name_of_gene_id(accession)
    genes_list.append(name)

In [25]:
len(ensembl_list), len(genes_list)

(11783, 11783)

In [26]:
limma_df.index = genes_list

In [ ]:
limma_df

In [29]:
limma_df.to_csv('RSEM_RES/Limma_results_2.txt', sep='\t', index=True)

# EBSeq processing

In [60]:
ebseq_df = pd.read_csv('EBSeq_PPMat.txt', sep='\t', skiprows=(0), header=(0))

In [61]:
ebseq_df

,PPEE,PPDE
ENSG00000000419,0.989269,0.010731
ENSG00000000457,0.995765,0.004235
ENSG00000000460,0.986121,0.013879
ENSG00000000938,0.998493,0.001507
ENSG00000000971,0.842363,0.157637
...,...,...
ENSG00000288349,0.828411,0.171589
ENSG00000288380,0.994968,0.005032
ENSG00000288398,0.975284,0.024716
ENSG00000288473,0.983714,0.016286


In [62]:
ensembl_ebseq = list(ebseq_df.index)

In [63]:
# Query pyensembl to get gene symbols (if exists and unique)
genes_list = []
for accession in ensembl_ebseq:
    name = data.gene_name_of_gene_id(accession)
    genes_list.append(name)

In [64]:
len(ensembl_ebseq), len(genes_list)

(22902, 22902)

In [65]:
ebseq_df.index = genes_list

In [66]:
ebseq_df.to_csv('EBSeq_PPMat.txt', sep='\t', index=True)